## Exploratory Data Analysis of Fitness & Workout Dataset:
---

### Author: Caitlin Driscoll
### Date: October 2025
### Source: Kaggle
---

In [5]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('../Data/fitness_and_workout_dataset.csv')

print("=" * 60)
print("ORIGINAL DATASET")
print("=" * 60)
print(f"Shape: {df.shape}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nData Types:\n{df.dtypes}")

# ============================================================================
# 1. HANDLE MISSING VALUES
# ============================================================================

print("\n" + "=" * 60)
print("STEP 1: HANDLING MISSING VALUES")
print("=" * 60)

# Description: Fill with 'No description available'
df['description'].fillna('No description available', inplace=True)
print(f"✓ Filled {df['description'].isnull().sum()} missing descriptions")

# Equipment: Fill with mode (most common value)
equipment_mode = df['equipment'].mode()[0]
df['equipment'].fillna(equipment_mode, inplace=True)
print(f"✓ Filled {df['equipment'].isnull().sum()} missing equipment with '{equipment_mode}'")

# Program length: Fill with median
program_length_median = df['program_length'].median()
df['program_length'].fillna(program_length_median, inplace=True)
print(f"✓ Filled missing program_length with median: {program_length_median}")

# Date columns: Drop rows with missing dates (only 1-2 rows)
rows_before = len(df)
df.dropna(subset=['created', 'last_edit'], inplace=True)
rows_dropped = rows_before - len(df)
print(f"✓ Dropped {rows_dropped} rows with missing dates")

# ============================================================================
# 2. PARSE LIST-FORMATTED COLUMNS (level and goal)
# ============================================================================

print("\n" + "=" * 60)
print("STEP 2: PARSING LIST-FORMATTED COLUMNS")
print("=" * 60)

def parse_list_column(value):
    """Safely parse string representation of list"""
    if pd.isna(value) or value == '[]':
        return []
    try:
        # Use ast.literal_eval to safely evaluate string as Python literal
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return []

# Parse level and goal columns
df['level_list'] = df['level'].apply(parse_list_column)
df['goal_list'] = df['goal'].apply(parse_list_column)

print(f"✓ Parsed 'level' column into 'level_list'")
print(f"✓ Parsed 'goal' column into 'goal_list'")

# Check for empty lists
empty_levels = (df['level_list'].apply(len) == 0).sum()
empty_goals = (df['goal_list'].apply(len) == 0).sum()
print(f"\nEmpty lists found:")
print(f"  - level_list: {empty_levels} rows")
print(f"  - goal_list: {empty_goals} rows")

# ============================================================================
# 3. CREATE BINARY COLUMNS FOR LEVELS
# ============================================================================

print("\n" + "=" * 60)
print("STEP 3: CREATING BINARY LEVEL COLUMNS")
print("=" * 60)

# Get all unique levels
all_levels = set()
for levels in df['level_list']:
    all_levels.update(levels)

print(f"Unique levels found: {sorted(all_levels)}")

# Create binary columns for each level
for level in sorted(all_levels):
    col_name = f'level_{level.lower().replace(" ", "_")}'
    df[col_name] = df['level_list'].apply(lambda x: 1 if level in x else 0)
    print(f"✓ Created '{col_name}' (sum: {df[col_name].sum()})")

# ============================================================================
# 4. CREATE BINARY COLUMNS FOR GOALS
# ============================================================================

print("\n" + "=" * 60)
print("STEP 4: CREATING BINARY GOAL COLUMNS")
print("=" * 60)

# Get all unique goals
all_goals = set()
for goals in df['goal_list']:
    all_goals.update(goals)

print(f"Unique goals found: {sorted(all_goals)}")

# Create binary columns for each goal
for goal in sorted(all_goals):
    col_name = f'goal_{goal.lower().replace(" ", "_").replace("&", "and")}'
    df[col_name] = df['goal_list'].apply(lambda x: 1 if goal in x else 0)
    print(f"✓ Created '{col_name}' (sum: {df[col_name].sum()})")

# ============================================================================
# 5. CONVERT DATE COLUMNS
# ============================================================================

print("\n" + "=" * 60)
print("STEP 5: CONVERTING DATE COLUMNS")
print("=" * 60)

df['created'] = pd.to_datetime(df['created'])
df['last_edit'] = pd.to_datetime(df['last_edit'])

# Create additional date features
df['created_year'] = df['created'].dt.year
df['created_month'] = df['created'].dt.month
df['days_since_creation'] = (datetime.now() - df['created']).dt.days
df['days_since_edit'] = (datetime.now() - df['last_edit']).dt.days

print(f"✓ Converted date columns to datetime")
print(f"✓ Created 'created_year' and 'created_month'")
print(f"✓ Created 'days_since_creation' and 'days_since_edit'")

# ============================================================================
# 6. ENCODE EQUIPMENT COLUMN
# ============================================================================

print("\n" + "=" * 60)
print("STEP 6: ENCODING EQUIPMENT")
print("=" * 60)

equipment_mapping = {
    'Full Gym': 3,
    'Garage Gym': 2,
    'Dumbbell Only': 1,
    'At Home': 0
}

df['equipment_encoded'] = df['equipment'].map(equipment_mapping)
print(f"Equipment encoding:")
for equip, code in equipment_mapping.items():
    count = (df['equipment'] == equip).sum()
    print(f"  {equip}: {code} (count: {count})")

# ============================================================================
# 7. CREATE AGGREGATE FEATURES
# ============================================================================

print("\n" + "=" * 60)
print("STEP 7: CREATING AGGREGATE FEATURES")
print("=" * 60)

# Count of levels and goals
df['num_levels'] = df['level_list'].apply(len)
df['num_goals'] = df['goal_list'].apply(len)

# Total program hours
df['total_program_hours'] = (df['program_length'] * df['time_per_workout']) / 60

# Exercises per week
df['exercises_per_week'] = df['total_exercises'] / df['program_length']

print(f"✓ Created 'num_levels' (mean: {df['num_levels'].mean():.2f})")
print(f"✓ Created 'num_goals' (mean: {df['num_goals'].mean():.2f})")
print(f"✓ Created 'total_program_hours' (mean: {df['total_program_hours'].mean():.2f})")
print(f"✓ Created 'exercises_per_week' (mean: {df['exercises_per_week'].mean():.2f})")

# ============================================================================
# 8. FINAL CLEANUP AND VALIDATION
# ============================================================================

print("\n" + "=" * 60)
print("STEP 8: FINAL CLEANUP")
print("=" * 60)

# Remove rows where level_list or goal_list is empty
df_clean = df[df['level_list'].apply(len) > 0].copy()
df_clean = df_clean[df_clean['goal_list'].apply(len) > 0].copy()

rows_removed = len(df) - len(df_clean)
print(f"✓ Removed {rows_removed} rows with empty level or goal lists")

# Strip whitespace from title
df_clean['title'] = df_clean['title'].str.strip()

print(f"\n✓ Data cleaning complete!")

# ============================================================================
# 9. SUMMARY
# ============================================================================

print("\n" + "=" * 60)
print("CLEANED DATASET SUMMARY")
print("=" * 60)
print(f"Final shape: {df_clean.shape}")
print(f"Columns: {len(df_clean.columns)}")
print(f"\nMissing values: {df_clean.isnull().sum().sum()}")
print(f"\nNew columns created:")

new_columns = [col for col in df_clean.columns if col not in df.columns]
for col in new_columns:
    print(f"  - {col}")

# ============================================================================
# 10. SAVE CLEANED DATA
# ============================================================================

print("\n" + "=" * 60)
print("SAVING CLEANED DATA")
print("=" * 60)

df_clean.to_csv('fitness_dataset_cleaned.csv', index=False)
print(f"✓ Saved to 'fitness_dataset_cleaned.csv'")

# Optional: Save a version with only key columns for analysis
key_columns = [
    'title', 'description', 'equipment', 'equipment_encoded',
    'program_length', 'time_per_workout', 'total_exercises',
    'num_levels', 'num_goals', 'total_program_hours', 'exercises_per_week',
    'created_year', 'created_month', 'days_since_creation'
] + [col for col in df_clean.columns if col.startswith('level_') or col.startswith('goal_')]

df_analysis = df_clean[key_columns].copy()
df_analysis.to_csv('fitness_dataset_analysis_ready.csv', index=False)
print(f"✓ Saved analysis-ready version to 'fitness_dataset_analysis_ready.csv'")

print("\n" + "=" * 60)
print("DATA PREVIEW")
print("=" * 60)
print(df_clean.head())


ORIGINAL DATASET
Shape: (2598, 10)

Missing Values:
title               0
description         4
level               0
goal                0
equipment           1
program_length      1
time_per_workout    0
total_exercises     0
created             1
last_edit           2
dtype: int64

Data Types:
title                object
description          object
level                object
goal                 object
equipment            object
program_length      float64
time_per_workout    float64
total_exercises       int64
created              object
last_edit            object
dtype: object

STEP 1: HANDLING MISSING VALUES
✓ Filled 0 missing descriptions
✓ Filled 0 missing equipment with 'Full Gym'
✓ Filled missing program_length with median: 8.0
✓ Dropped 2 rows with missing dates

STEP 2: PARSING LIST-FORMATTED COLUMNS
✓ Parsed 'level' column into 'level_list'
✓ Parsed 'goal' column into 'goal_list'

Empty lists found:
  - level_list: 8 rows
  - goal_list: 8 rows

STEP 3: CREATING BINARY L